# Tangential average of pipe flow data
In this tutorial, we compute the tangential average of the unstructured pipe flow data. The original data is in Cartesian coordinates, $(x, y, z)$. Here, we average in the circumferential direction and interpolate onto a structured $r-z$ grid.

In [1]:
# Read data same manner as reading_cs

import pyvista as pv
import numpy as np
fn = 'case/postprocessing/RESULTS_FLUID_DOMAIN.case'

reader = pv.get_reader(fn)
reader.set_active_time_value(300)

data = reader.read()['Fluid domain']

In [2]:
# Cell cell center coordinates
x, y, z = data.cell_centers().points.T


In [3]:
# Compute radius 
r = np.sqrt(x*x + y*y)

coords = np.stack([r,np.zeros(data.n_cells),z],axis=1)

#Create poly data point cloud with associated cell arrays
plane = pv.PolyData(coords)
for name in data.cell_data.keys():
    plane.point_data[name] = data.cell_data[name]

In [7]:
# Plot point cloud (zoom to see the points)
p = pv.Plotter(window_size=(500,500))

p.add_mesh(plane.points)

p.view_zx()
p.show()

Widget(value='<iframe src="http://localhost:33849/index.html?ui=P_0x733e6c385190_2&reconnect=auto" class="pyvi…

In [5]:
# Create structured r-z grid

zmin = data.bounds[4]
zmax = data.bounds[5]
z= np.linspace(zmin, zmax, 201)

# using CHAPSim's stretching function to skew mesh towards wall
X2= np.linspace(0, 1, 101)
STR = 1.4
r = np.tanh(X2*STR)/np.tanh(STR)

R, Z = np.meshgrid(r, z, indexing='ij')

# Create vtkStructured Grid
s = pv.StructuredGrid(R, np.zeros_like(R),Z)

# Interpolate point cloud onto structured grid
s = s.interpolate(plane, radius=0.05, strategy='closest_point')

s.point_data

# We could also use functions from the scipy.interpolate module 
# Bypassing the StructuredGrid altogether

pyvista DataSetAttributes
Association     : POINT
Active Scalars  : mpi_rank_id
Active Vectors  : None
Active Texture  : None
Active Normals  : None
Contains arrays :
    mpi_rank_id             float32    (20301,)             SCALARS
    Velocity                float32    (20301, 3)
    Pressure                float32    (20301,)
    TurbVisc                float32    (20301,)
    CourantNb               float32    (20301,)
    FourierNb               float32    (20301,)
    total_pressure          float32    (20301,)
    U_mean                  float32    (20301, 3)
    UU_mean                 float32    (20301, 6)
    P_mean                  float32    (20301,)

In [6]:
# Plot both original data and circumferentially averaged data
p = pv.Plotter()
p.add_mesh(s,
           scalars='U_mean',
           cmap='bwr')

p.add_mesh(data.slice(normal='y',origin=(0,0,0)).translate([-1.2,0,0]),
           scalars='U_mean',
           cmap='bwr')

p.view_zx()
p.show()

Widget(value='<iframe src="http://localhost:33849/index.html?ui=P_0x733e6c312390_1&reconnect=auto" class="pyvi…